In [4]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import requests
import os

In [153]:
#URL of the PDF file
# url = 'https://ieeexplore.ieee.org/document/9574048'

#Download the papar here.
pdf = 'deep_learning_paper.pdf'

In [154]:
papers = []
loader = PyPDFLoader(pdf)
document = loader.load()
papers.append(document)

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
chunked_papers = []
for paper in papers:
    texts = text_splitter.split_documents(paper)
    chunked_papers.append(texts)
    print(f"chunked_papers length is: {len(texts)}")


chunked_papers length is: 63


In [146]:
texts

[Document(page_content='Deep Learning for Self-tuning of Control systems\nJunaid Farooq\nDepartment of Electrical Engineering\nNational Institute of Technology Srinagar\nSrinagar, India\njunaid phd017@nitsri.ac.inMohammad Abid Bazaz\nDepartment of Electrical Engineering\nNational Institute of Technology Srinagar\nSrinagar, India\nabid@nitsri.ac.in\nAbstract —This paper proposes an innovative two-dimensional\nmulti-layered deep neural network (DNN) to achieve adaptive,', metadata={'source': 'deep_learning_paper.pdf', 'page': 0}),
 Document(page_content='physics-informed, model-free and data-based control of stochas-\ntic, sensitive and highly nonlinear systems. The algorithm design\nexploits the DNN features of adaptive learning, inference of latent\nvariables and time-series prediction to update the controller\nparameters on-the-run in real-time while compensating for the\nloop delays at the same time in addition to the system processing\ntime. The proposed deep learning based self-tun

In [94]:
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from tqdm.autonotebook import tqdm
import pinecone
import os

In [95]:
# Get these API
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV')

In [96]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [97]:
pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_API_ENV)

In [98]:
# pinecone.create_index("quickstart", dimension=8, metric="euclidean", pod_type="p1")
pinecone.list_indexes

<function pinecone.manage.list_indexes()>

In [99]:
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_API_ENV
)
index_name = "your_index_name"

In [155]:
chunked_papers

[[Document(page_content='Deep Learning for Self-tuning of Control systems\nJunaid Farooq\nDepartment of Electrical Engineering\nNational Institute of Technology Srinagar\nSrinagar, India\njunaid phd017@nitsri.ac.inMohammad Abid Bazaz\nDepartment of Electrical Engineering\nNational Institute of Technology Srinagar\nSrinagar, India\nabid@nitsri.ac.in\nAbstract —This paper proposes an innovative two-dimensional\nmulti-layered deep neural network (DNN) to achieve adaptive,', metadata={'source': 'deep_learning_paper.pdf', 'page': 0}),
  Document(page_content='physics-informed, model-free and data-based control of stochas-\ntic, sensitive and highly nonlinear systems. The algorithm design\nexploits the DNN features of adaptive learning, inference of latent\nvariables and time-series prediction to update the controller\nparameters on-the-run in real-time while compensating for the\nloop delays at the same time in addition to the system processing\ntime. The proposed deep learning based self-t

In [101]:
for chunks in chunked_papers[0]:
    # dummy = [chunks.page_content for chunk in chunks]
    Pinecone.from_texts([chunks.page_content for chunk in chunks], embeddings, index_name=index_name)

In [102]:
vectorstore = Pinecone.from_existing_index(index_name=index_name, embedding=embeddings)

In [103]:
from langchain.chains import ConversationalRetrievalChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI

In [112]:
# Create the chain

qa = ConversationalRetrievalChain.from_llm(
    llm=OpenAI(openai_api_key=OPENAI_API_KEY, temperature=0),
    retriever=vectorstore.as_retriever(),
    return_source_documents=True,
)

In [113]:
# Initialize chat history list
chat_history = []

In [156]:
# Notice how the chatbot has memory below and can understand a question asked based on the information provided in previous questions.
query = "Who is the name of the proposed algorithm?"
result = qa({"question": query, "chat_history": chat_history})
result["answer"]

' The proposed algorithm is a deep learning based self-tuning algorithm.'

In [119]:
# Add the answer to the chat history
chat_history.append((query, result["answer"]))


In [120]:
query = "What are its main components?"
result = qa({"question": query, "chat_history": chat_history})
result["answer"]

' The main components of the deep learning based self-tuning algorithm are adaptive learning, inference of latent variables, time-series prediction, and on-the-run optimization of controller parameters.'

In [121]:
# Add the answer to the chat history
chat_history.append((query, result["answer"]))

In [122]:
query = "How does it tackle the problem of delay?"
result = qa({"question": query, "chat_history": chat_history})
result["answer"]

' The deep learning based self-tuning algorithm compensates for the loop delays at the same time in addition to the system processing time.'

In [123]:
# Add the answer to the chat history
chat_history.append((query, result["answer"]))

In [124]:
query = "Who are the authors of this paper?"
result = qa({"question": query, "chat_history": chat_history})
result["answer"]

' Junaid Farooq and Mohammad Abid Bazaz.'

In [125]:
# Add the answer to the chat history
chat_history.append((query, result["answer"]))

In [126]:
query = "Who are their affiliations and to which institutions do they belong?"
result = qa({"question": query, "chat_history": chat_history})
result["answer"]

' Junaid Farooq is affiliated with the Department of Electrical Engineering at the National Institute of Technology Srinagar in Srinagar, India. Mohammad Abid Bazaz is also affiliated with the Department of Electrical Engineering at the National Institute of Technology Srinagar in Srinagar, India.'

In [127]:
# Add the answer to the chat history
chat_history.append((query, result["answer"]))

In [128]:
query = "How can I contact them?"
result = qa({"question": query, "chat_history": chat_history})
result["answer"]

' You can contact Junaid Farooq at junaid phd017@nitsri.ac.in and Mohammad Abid Bazaz at abid@nitsri.ac.in.'

In [129]:
# Add the answer to the chat history
chat_history.append((query, result["answer"]))

In [135]:
query = "What is the main idea of their paper?"
result = qa({"question": query, "chat_history": chat_history})
result["answer"]

' The paper proposes an innovative two-dimensional multi-layered deep neural network (DNN) to achieve adaptive reduction techniques.'

In [136]:
# Add the answer to the chat history
chat_history.append((query, result["answer"]))

In [137]:
query = "Your are wrong"
result = qa({"question": query, "chat_history": chat_history})
result["answer"]

' No, the information you were given is correct.'

In [138]:
# Add the answer to the chat history
chat_history.append((query, result["answer"]))

In [139]:
query = "The paper proposes an innovative two-dimensional multi-layered deep neural network (DNN) to achieve self-tuning of control systems."
result = qa({"question": query, "chat_history": chat_history})
result["answer"]

' The paper proposes an innovative two-dimensional multi-layered deep neural network (DNN) to achieve adaptive, physics-informed, model-free and data-based control of stochastic, sensitive and highly nonlinear systems. The algorithm design exploits the DNN features of adaptive learning, inference of latent variables and time-series prediction to update the controller parameters on-the-run in real-time while compensating for the loop delays at the same time in addition to the system processing time.'

In [140]:
# Add the answer to the chat history
chat_history.append((query, result["answer"]))

In [141]:
query = "Your are right now. That is like a good boy."
result = qa({"question": query, "chat_history": chat_history})
result["answer"]

' Yes, the information given is correct.'

In [142]:
# Add the answer to the chat history
chat_history.append((query, result["answer"]))

In [143]:
query = "What is the main idea of their paper?"
result = qa({"question": query, "chat_history": chat_history})
result["answer"]

' The paper proposes a deep learning based self-tuning algorithm to address four major challenges in design and modeling of control systems: loop delays, nonlinearity, stochasticity, and sensitivity. The algorithm design exploits the DNN features of adaptive learning, inference of latent variables and time-series prediction to update the controller parameters on-the-run in real-time while compensating for the loop delays at the same time in addition to the system processing time.'

In [144]:
# Add the answer to the chat history
chat_history.append((query, result["answer"]))

In [145]:
query = "Can you rephrase the above answer?"
result = qa({"question": query, "chat_history": chat_history})
result["answer"]

' The deep learning based self-tuning algorithm can address the four major challenges in design and modeling of control systems by exploiting the DNN features of adaptive learning, inference of latent variables and time-series prediction to update the controller parameters on-the-run in real-time while compensating for the loop delays at the same time in addition to the system processing time.'

In [147]:
# Add the answer to the chat history
chat_history.append((query, result["answer"]))

In [149]:
query = "What are the 4 main challenges discussed?"
result = qa({"question": query, "chat_history": chat_history})
result["answer"]

' The four major challenges addressed by the deep learning based self-tuning algorithm are loop delays, nonlinearity, stochasticity, and sensitivity.'

In [150]:
# Add the answer to the chat history
chat_history.append((query, result["answer"]))

In [152]:
query = "Which among these leads to mismatch between the actual output and desired output?"
result = qa({"question": query, "chat_history": chat_history})
result["answer"]

' Loop delays, nonlinearity, stochasticity, and sensitivity lead to a mismatch between the actual output and desired output in the four major challenges addressed by the deep learning based self-tuning algorithm.'